In [5]:
import pandas as pd
import numpy as np
import os
import re
import io
import functools
import math
from sklearn.cluster import *
import matplotlib.pyplot as plt

In [6]:
k = 2
l = math.ceil(math.log(k, 2))

w = 1000

low = 0.02
high = 0.95

In [20]:
def standardize(word):
    word = word.lower()
    return word
    
def create_dict(text_documents):
    dictionary = []
    for doc in text_documents:
        lines = doc.split("\n")
        for line in lines:
            words = re.split('[,.;:!? \t(){}]', line)
            for i, word in enumerate(words):
                if i >= w:
                    break
                word = standardize(word)
                if word == '':
                    continue
                if word not in dictionary:
                    dictionary.append(word)
    return dictionary

def text_features_extractor(text):
    features = dict(zip(dictionary, np.zeros(len(dictionary), dtype='int')))
    keys = features.keys()
    lines = text.split("\n")
    for line in lines:
        words = re.split('[,.;:!? \t]', line)
        for i, word in enumerate(words):
            word = standardize(word)
            if word in keys:
                features[word] += 1
#     r.close()
    return list(features.values())

In [21]:
dataset = 'Articles.csv'
data = pd.read_csv(dataset)
data1 = data.iloc[:100,:]
data2 = data.iloc[1800:1900,:]
data = pd.concat([data1, data2])
dictionary = create_dict(data['Article'])
labels = data['NewsType']
labels_head = np.unique(labels.to_numpy())
num_labels = len(labels_head)

In [22]:
dataset = 'index_articles.csv'
data = pd.read_csv(dataset)
dictionary = create_dict(data['content'])
labels = data['category']
labels_head = np.unique(labels.to_numpy())
num_labels = len(labels_head)

In [23]:
# A = data['content'].apply(lambda row: text_features_extractor(row)).values.tolist()
# A = np.transpose(A)
# # print(A)

In [24]:
A = data['content'].apply(lambda row: text_features_extractor(row)).values.tolist()
A = np.transpose(A)
# # print(A)

In [25]:
A = np.array(A)
print(A.shape)

(7454, 64)


In [26]:
# filtracija A, izbacivanje rijetkih i cestih rijeci
df = pd.DataFrame(A)
df2 = df.applymap(lambda x: 1 if x != 0 else 0)
counts = df2.apply(lambda x: sum(x) / len(labels), axis=1)
selected = [low < count < high for count in counts]
df = df.loc[selected, :]
A = df.to_numpy()
print(A)

[[1 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 [3 1 2 ... 2 6 2]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 1 1]]


In [27]:
print(A.shape)

(1869, 64)


In [28]:
D1_diag = np.sum(A, axis=1)
D1 = np.diag(D1_diag)

In [29]:
D2_diag = np.sum(A, axis=0)
D2 = np.diag(D2_diag)

In [30]:
D1_inv = np.diag(list(map(lambda x: 0 if x == 0 else 1/(np.sqrt(x)), D1_diag)))
D2_inv = np.diag(list(map(lambda x: 0 if x == 0 else 1/(np.sqrt(x)), D2_diag)))

In [31]:
An = D1_inv @ A @ D2_inv

In [32]:
u, s, vt = np.linalg.svd(An)
v = np.transpose(vt)

In [33]:
U = u[:, 1:(1+l)]
V = v[:, 1:(1+l)]
print(U.shape)
print(V.shape)

(1869, 1)
(64, 1)


In [34]:
Z = np.block([[D1_inv @ U], [D2_inv @ V]])
# Z = Z.tolist()
# Z = list(map(lambda t: t[0], Z))
# # print(Z)
# Z = np.array(Z).reshape(-1, 1)

In [37]:
kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300, random_state=42)
kmeans.fit(Z)
predictions = kmeans.predict(Z[-len(labels):])

In [38]:
table = np.zeros((k, num_labels), dtype='int')
table = pd.DataFrame(table, columns=labels_head)

for i, pred in enumerate(predictions):
    label = labels.iloc[i]
#     print("{}-{}".format(pred, label))
    table[label][pred] += 1
print(table)

   sport  vijesti
0      0        1
1     31       32
